In [1]:
import matplotlib.pyplot as plt
from itertools import combinations, permutations, product, chain
import numpy as np
import gc, csv

In [37]:
'''
Function for Sharpe ration calculation for various number of drivers in one calculation

Input is 
- an array of possibilities for all the drivers to score
- an array of money stakes for each driver (generates later to gen a distribution)
- an array of bookmaker coefficients for each driver

The dimentions obviously should be the same and equal to number on drivers in a run

'''

def sharpe(pos, mon, koef):

    p_temp = []
    m_temp = []
    k_temp = []
    
    # Calculating a list of possible intersections for each instance
    for i in range(len(pos)):
        p_temp.append([pos[i], 1 - pos[i]])
        m_temp.append([mon[i], mon[i] - mon[i]])
        k_temp.append([koef[i], koef[i] - koef[i]])

    # Generating all the combinations
    p = list(product(*p_temp))
    m = list(product(*m_temp))
    k = list(product(*k_temp))

    # Calculating probabilities for each income
    p_list = []
    for i in range(len(p)):
        p_item = p[i][0]
        for j in range(1, len(p[0])):
            p_item *= p[i][j]
        p_list.append(p_item)

    # Overall stake sum
    cost = sum(mon)

    # Profit weighted with probability for each income
    prof = []
    for i in range(len(p)):
        prof.append(sum([(m[i][j] * k[i][j]) for j in range(len(m[i]))]) - cost)

    # Mean expectation
    mex = sum([pr * p for pr, p in zip(prof, p_list)])

    # Dispersion
    disp = 0
    for i in range(len(p)):
        disp += (len(mon) / (len(mon) - 1)) * p_list[i] * ((prof[i] - mex) ** 2)

    # Returning Sharpe ratio
    return mex / disp

In [25]:
sharpe([0.6, 0.7], [0.4, 0.6], [2, 1.1])

ME is -0.058000000000000024
Dispersion is 0.24507600000000002


-0.23666128058235006

In [26]:
sharpe([0.6, 0.7, 0.8], [0.4, 0.6, 0], [2, 1.1, 1.5])

ME is -0.058
Dispersion is 0.24507600000000002


-0.23666128058234995

In [38]:
drivers = [['Kvyat', 2, 0.6], ['Lewis', 1.1, 0.9], ['Seb', 1.2, 0.85], ['Kubica', 5, 0.3], ['Sergio', 1.8, 0.7],
          ['Filipe', 1.6, 0.65], ['Marcus', 3.5, 0.4]]

In [68]:
def challenger(drivers):
    
    gc.collect()
    
    combs = []
    for num_drivers in [2, 3, 4, 5, 6]:
        combs.append(list(combinations(drivers, num_drivers)))

    combs = list(chain(*combs))

    overall_max = [0]
    stakes_no = 0
    for run in combs:
        
        stakes_no_old = stakes_no
        stakes_no = len(run)

        if stakes_no_old != stakes_no:
            
            mon = np.genfromtxt(r'./Money_sets/money_set_' + str(len(run)) + '.txt', delimiter=';')

        pos = []
        koef = []
        driver = []

        for i in range(len(run)):
            pos.append(run[i][2])
            koef.append(run[i][1])
            driver.append(run[i][0])

        local_max = [0]
        for m in mon:
            sh = sharpe(pos, m, koef)
            if sh > local_max[0]:
                local_max = [sh]
                local_max.append(m)
                local_max.append(driver)
        print(f"For pair {driver} maximum profit is {round(local_max[0],2)} at money spent like {local_max[1]}")
        if local_max[0] > overall_max[0]:
            overall_max = [local_max[0]]
            overall_max.append(local_max[2])
            overall_max.append(local_max[1])

    print(f"Overall maximum is for pair {overall_max[1]} maximum profit is {round(overall_max[0],2)} at money spent like {overall_max[2]}")

In [67]:
challenger(drivers)

For pair ['Kvyat', 'Lewis'] maximum profit is 0.19 at money spent like [0.4 0.6]
For pair ['Kvyat', 'Seb'] maximum profit is 0.21 at money spent like [0.3 0.7]
For pair ['Kvyat', 'Kubica'] maximum profit is 0.16 at money spent like [0.8 0.2]
For pair ['Kvyat', 'Sergio'] maximum profit is 0.3 at money spent like [0.4 0.6]
For pair ['Kvyat', 'Filipe'] maximum profit is 0.16 at money spent like [0.5 0.5]
For pair ['Kvyat', 'Marcus'] maximum profit is 0.18 at money spent like [0.7 0.3]
For pair ['Lewis', 'Seb'] maximum profit is 0.06 at money spent like [0.2 0.8]
For pair ['Lewis', 'Kubica'] maximum profit is 0.16 at money spent like [0.8 0.2]
For pair ['Lewis', 'Sergio'] maximum profit is 0.33 at money spent like [0.6 0.4]
For pair ['Lewis', 'Filipe'] maximum profit is 0.04 at money spent like [0.4 0.6]
For pair ['Lewis', 'Marcus'] maximum profit is 0.19 at money spent like [0.8 0.2]
For pair ['Seb', 'Kubica'] maximum profit is 0.18 at money spent like [0.8 0.2]
For pair ['Seb', 'Sergio']

For pair ['Kvyat', 'Lewis', 'Seb', 'Kubica', 'Sergio'] maximum profit is 0.86 at money spent like [0.1 0.3 0.3 0.1 0.2]
For pair ['Kvyat', 'Lewis', 'Seb', 'Kubica', 'Filipe'] maximum profit is 0.63 at money spent like [0.2 0.3 0.3 0.1 0.1]
For pair ['Kvyat', 'Lewis', 'Seb', 'Kubica', 'Marcus'] maximum profit is 0.73 at money spent like [0.2 0.3 0.3 0.1 0.1]
For pair ['Kvyat', 'Lewis', 'Seb', 'Sergio', 'Filipe'] maximum profit is 0.84 at money spent like [0.1 0.3 0.2 0.2 0.2]
For pair ['Kvyat', 'Lewis', 'Seb', 'Sergio', 'Marcus'] maximum profit is 0.91 at money spent like [0.1 0.4 0.1 0.3 0.1]
For pair ['Kvyat', 'Lewis', 'Seb', 'Filipe', 'Marcus'] maximum profit is 0.7 at money spent like [0.2 0.3 0.3 0.1 0.1]
For pair ['Kvyat', 'Lewis', 'Kubica', 'Sergio', 'Filipe'] maximum profit is 0.81 at money spent like [0.1 0.4 0.1 0.3 0.1]
For pair ['Kvyat', 'Lewis', 'Kubica', 'Sergio', 'Marcus'] maximum profit is 0.87 at money spent like [0.1 0.4 0.1 0.3 0.1]
For pair ['Kvyat', 'Lewis', 'Kubica